### Лабораторная работа 5.

#### Часть 1

4.1 Написать функцию формирования порождающей матрицы кода Рида Маллера (r,m) в каноническом виде для произвольно заданных r и m.

4.2. Реализовать алгоритм мажоритарного декодирования для кода Рида Маллера.

4.3. Провести экспериментальную проверку алгоритма декодирования для кода Рида-Маллера (2,4).

In [11]:
import numpy as np
from itertools import combinations
import random
import math

def generate_basis(n, m):
    result = []
    for i in range(n):
        binary = bin(i)[2:].zfill(m)
        result.append(binary[::-1])
    return result

def generate_vector_orders(r, m):
    elements = list(range(m))
    result = []
    for i in range(r + 1):
        for combo in sorted(combinations(elements, i), reverse=True):
            result.append(list(combo))
    return result

def construct_rm_matrix(r, m):
    n = 2 ** m
    basis = generate_basis(n, m)
    vector_orders = generate_vector_orders(r, m)
    matrix = np.zeros((len(vector_orders), n), dtype=int)

    for row in range(matrix.shape[0]):
        for col in range(matrix.shape[1]):
            if all(basis[col][index] == '0' for index in vector_orders[row]):
                matrix[row][col] = 1

    print("Intermediate output: Generating matrix of Reed-Muller code")
    print(matrix)
    print("Intermediate output: Basis order")
    print(basis)
    print("Intermediate output: Vector order")
    print(vector_orders)

    return matrix, basis, vector_orders

def find_complement(m, subset):
    full_set = list(range(m))
    return [item for item in full_set if item not in subset]

def extract_Hj(matrix, basis, vectors, subset, m):
    Hj = []
    complement = list(subset)
    complement_index = vectors.index(complement if complement != list(range(m)) else [])
    for i, val in enumerate(matrix[complement_index]):
        if val == 1:
            Hj.append(basis[i])
    return Hj

def compute_vector(complement, basis, hj):
    vector = []
    for position in basis:
        vector.append(1 if all(position[j] == hj[j] for j in complement) else 0)
    return vector

def majority_decode(received, m, basis, r, matrix, vectors):
    decoded = {}
    for length in range(r, -1, -1):
        if length == r:
            modified = received
        else:
            for key, value in sorted(decoded.items()):
                if len(key) == length + 1 and value == 1:
                    temp = np.array(modified) ^ matrix[vectors.index(list(key))]
                    modified = temp.tolist()
                    break

        for subset in combinations(range(m), length):
            subset_complement = find_complement(m, subset)
            Hj = extract_Hj(matrix, basis, vectors, subset, m)
            count_one, count_zero = 0, 0

            for hj in Hj:
                V = compute_vector(subset_complement, basis, hj)
                count = sum((v or mod) for v, mod in zip(V, modified))

                if subset_complement == list(range(m)):
                    decoded[tuple(subset)] = 0
                    break
                elif count > 2 ** (m - length - 1):
                    decoded[tuple(subset)] = 1
                    break
                elif count < 2 ** (m - length - 1):
                    decoded[tuple(subset)] = 0
                    break
    return decoded

def test_with_errors(matrix, r, basis, vectors, t):
    m = int(math.log2(matrix.shape[1]))
    row = matrix.shape[0]

    idx = random.randint(0, row - 1)
    word = matrix[idx][:row]
    sent = np.dot(word, matrix) % 2

    print(f"Original message: {word}")
    print(f"Transmitted message: {sent}")
    for i in range(t):
        sent[i] ^= 1
    print(f"Received message with errors: {sent}")

    decoded_data = majority_decode(sent.tolist(), m, basis, r, matrix, vectors)
    decoded_message = [decoded_data[tuple(key)] for key in sorted(decoded_data.keys(), reverse=True)]
    print(f"Modified message: {decoded_message}")
    try:
        print(f"Decoded message: {np.dot(decoded_message, matrix) % 2}")
    except:
        print("Error occurred. Retransmission required.")

r, m = 2, 4
rm_matrix, basis_order, vector_order = construct_rm_matrix(r, m)
t_values = [1, 2]

for t in t_values:
    print(f"Decoding test RM({r}, {m}) with t = {t}")
    test_with_errors(rm_matrix, r, basis_order, vector_order, t)
    if t != t_values[-1]:
        print()


Intermediate output: Generating matrix of Reed-Muller code
[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0]
 [1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0]
 [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0]]
Intermediate output: Basis order
['0000', '1000', '0100', '1100', '0010', '1010', '0110', '1110', '0001', '1001', '0101', '1101', '0011', '1011', '0111', '1111']
Intermediate output: Vector order
[[], [3], [2], [1], [0], [2, 3], [1, 3], [1, 2], [0, 3], [0, 2], [0, 1]]
Decoding test RM(2, 4) with t = 1
Original message: [1 1 1 1 0 0 0 0 0 0 0]
Transmitted message: [0 0 1 1 1 1 0 0 1 1 0 0 0 0 1 1]
Received message with errors: [1 0 1 1 1 1 0 0 1 1 0 0 0 0 1 1]
Modified message: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
Decoded message: [0 1 1 0 0 0 0 0 0 0 0